In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs

In [4]:
#połączenie się z przeglądarką Brave na silniku chromium
options = webdriver.ChromeOptions()
options.binary_location = "C:/Program Files (x86)/BraveSoftware/Brave-Browser/Application/brave.exe"
chrome_driver_binary = "chromedriver.exe"
driver = webdriver.Chrome(chrome_driver_binary, options=options)

server_name = []
gold_amount = []
driver.get("http://gp-sms.pl/")